In [1]:
# カレントディレクトリをリポジトリ直下にするおまじない
import os
import tensorflow as tf 
print(tf.__version__)
print("hello world")
print(os.getcwd().split('\\')[-1] )
while os.getcwd().split('\\')[-1] != 'deep_dialog_tutorial': os.chdir('..')
print('current dir:', os.getcwd())


2.6.0
hello world
transformer
current dir: C:\Users\konolab\Desktop\Research\deep_dialog_tutorial


In [2]:
import os
print(os.getcwd())
import tensorflow as tf
from deepdialog.transformer.transformer import Transformer
from deepdialog.transformer.preprocess.batch_generator import BatchGenerator

C:\Users\konolab\Desktop\Research\deep_dialog_tutorial


# Create Data

In [3]:
data_path = 'data\\natsume.txt'

In [4]:
batch_generator = BatchGenerator()
batch_generator.load(data_path)

虚子もそこが聞きたいので、わざわざ誘ったのである。


In [5]:
vocab_size = batch_generator.vocab_size

# Create Model

In [6]:
graph = tf.Graph()
with graph.as_default():
    transformer = Transformer(
        vocab_size=vocab_size,
        hopping_num=4,
        head_num=8,
        hidden_dim=512,
        dropout_rate=0.1,
        max_length=50,
    )
    transformer.build_graph()

C:\Users\konolab\anaconda3\envs\tensorflow-gpu\lib\site-packages\keras\engine\base_layer_v1.py:1684: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


Instructions for updating:
Use `tf.cast` instead.


# Create Training Graph

In [7]:
save_dir = 'tmp/learning/transformer/'
log_dir = os.path.join(save_dir, 'log')
ckpt_path = os.path.join(save_dir, 'checkpoints/model.ckpt')

os.makedirs(log_dir, exist_ok=True)

In [8]:
with graph.as_default():
    global_step = tf.compat.v1.train.get_or_create_global_step()
    
    learning_rate = tf.compat.v1.placeholder(dtype=tf.float32, name='learning_rate')
    optimizer = tf.compat.v1.train.AdamOptimizer(
        learning_rate=learning_rate,
        beta2=0.98,
    )
    #TensorFlowには損失関数を最小限に抑えるために各変数をゆっくりと変更するオプティマイザが用意されている
    optimize_op = optimizer.minimize(transformer.loss, global_step=global_step)
# Tensor boardに出すためにデータをマージ
    summary_op = tf.compat.v1.summary.merge([
        tf.compat.v1.summary.scalar('train/loss', transformer.loss),
        tf.compat.v1.summary.scalar('train/acc', transformer.acc),
        tf.compat.v1.summary.scalar('train/learning_rate', learning_rate),
    ], name='train_summary')
    summary_writer = tf.compat.v1.summary.FileWriter(log_dir, graph)
    saver = tf.compat.v1.train.Saver()

# Train

In [9]:
max_step = 100000
batch_size = 128
max_learning_rate = 0.0001
warmup_step = 4000

In [10]:
def get_learning_rate(step: int) -> float:
    rate = min(step ** -0.5, step * warmup_step ** -1.5) / warmup_step ** -0.5
    return max_learning_rate * rate

In [11]:
with graph.as_default():
    sess = tf.compat.v1.Session()
    sess.run(tf.compat.v1.global_variables_initializer())
    step = 0

In [12]:
with graph.as_default():
    for batch in batch_generator.get_batch(batch_size=batch_size):
        feed = {
            **batch,
            learning_rate: get_learning_rate(step + 1),
        }
        _, loss, acc, step, summary = sess.run([optimize_op, transformer.loss, transformer.acc, global_step, summary_op], feed_dict=feed)
        summary_writer.add_summary(summary, step)
        
        if step % 100 == 0:
            print(f'{step}: loss: {loss},\t acc: {acc}')
            saver.save(sess, ckpt_path, global_step=step)
        if step == max_step:
            break

0: loss: 8.393471717834473,	 acc: 0.0
100: loss: 8.033754348754883,	 acc: 0.07527801394462585
200: loss: 7.603409290313721,	 acc: 0.08400169014930725
